##Group the compounds into the BRITE hierarchy...want to see how the different Kmeans groups do or do not fit with the BRITE hierarchy
Krista, 1 September 2015
Modify, 30 November 2015 to get the pathway BRITE hierarchy

This would probably make more sense as a script, but I did not bother to convert it. Will spit out a pickled file for other use.

In [19]:
import pandas as pd
import re
import glob
import cPickle as cpk

from IPython.core.debugger import Tracer #used this to step into the function and debug it, also need line with Tracer()() 


Now go in a read one BRITE file (originally for the compounds)

In [101]:
textC = re.compile('\d+')
mc = textC.search(line)
mc.group(0)
#so, now I have the map number...how do I get the rest of the line?

'01100'

In [116]:
textC = re.compile(r'(\d+)\s*(.*)$')
mC = textC.search(line)
print mC.group(1)
print mC.group(2)

01100
Metabolic pathways


In [134]:
def ReadBRITEfile(briteFile):
    forBrite = pd.DataFrame(columns = ['map','A','B','C','wholeThing'])
    # set up the expressions to match each level in the BRITE hierarchy
    
    textA = re.compile(r'(^A<b>)(.+)(</b>)\s*(.*)$')
    textB = re.compile(r'(^B)\s*(.*)$')
    textC = re.compile(r'(\d+)\s*(.*)$')
    #this relies on the fact that the rows are in order: A, with B subheadings, then C subheadings
    setA = []
    idxA = []

    setB = []
    setC = []

    with open(briteFile) as f:
        for idx,line in enumerate(f):
            if line[0] is not '#': #skip over the comments
                mA = textA.search(line) 
                mB = textB.search(line) 
                mC = textC.search(line) 
                if mA:
                    setA = mA.group(2)
                    #house cleaning (probably c)
                    idxA = idx
                    forBrite.loc[idx,'A'] = setA
                    forBrite.loc[idx,'wholeThing'] = line #using this as a double check for now
                    #forBrite.loc[idx,'map'] = mC.group(1)
                elif mB:
                    setB = mB.group(2)
                    forBrite.loc[idx,'A'] = setA
                    forBrite.loc[idx,'B'] = setB
                    forBrite.loc[idx,'wholeThing'] = line
                    #forBrite.loc[idx,'map'] = mC.group(1)
                elif mC:
                    #Tracer()()
                    setC = mC.group(2)
                    forBrite.loc[idx,'A'] = setA
                    forBrite.loc[idx,'B'] = setB
                    forBrite.loc[idx,'C'] = setC
                    forBrite.loc[idx,'wholeThing'] = line
                    forBrite.loc[idx,'map'] = mC.group(1)

        return forBrite

In [153]:
D = glob.glob('*keg.txt')
allBRITE=[]
for idx,nof in enumerate(D):
    allBRITE = ReadBRITEfile(nof)

In [138]:
type(allBRITE)

pandas.core.frame.DataFrame

In [146]:
allBRITE.loc[allBRITE['map']=='01100']

,map,A,B,C,wholeThing
10,01100,Metabolism,Global and overview maps,Metabolic pathways,C 01100 Metabolic pathways\n


In [147]:
allBRITE.loc[allBRITE['map']=='01100'].C

10    Metabolic pathways
Name: C, dtype: object

In [148]:
type(allBRITE.loc[allBRITE['map']=='01100'].C)

pandas.core.series.Series

In [137]:
allBRITE

,map,A,B,C,wholeThing
8,NaN,Metabolism,NaN,NaN,A<b>Metabolism</b>\n
9,NaN,Metabolism,Global and overview maps,NaN,B Global and overview maps\n
10,01100,Metabolism,Global and overview maps,Metabolic pathways,C 01100 Metabolic pathways\n
11,01110,Metabolism,Global and overview maps,Biosynthesis of secondary metabolites,C 01110 Biosynthesis of secondary metaboli...
12,01120,Metabolism,Global and overview maps,Microbial metabolism in diverse environments,C 01120 Microbial metabolism in diverse en...
13,01130,Metabolism,Global and overview maps,Biosynthesis of antibiotics,C 01130 Biosynthesis of antibiotics\n
14,01200,Metabolism,Global and overview maps,Carbon metabolism,C 01200 Carbon metabolism\n
15,01210,Metabolism,Global and overview maps,2-Oxocarboxylic acid metabolism,C 01210 2-Oxocarboxylic acid metabolism\n
16,01212,Metabolism,Global and overview maps,Fatty acid metabolism,C 01212 Fatty acid metabolism\n
17,01230,Metabolism,Global and overview maps,Biosynthesis of amino acids,C 01230 Biosynthesis of amino acids\n


In [151]:
#now...save all that so I don't have to do this everytime
cpk.dump(allBRITE, open('BRITE_pathwaysOnly.pickle', 'wb'))

In [152]:
cpk.load(open('BRITE_pathwaysOnly.pickle','rb'))

,map,A,B,C,wholeThing
8,NaN,Metabolism,NaN,NaN,A<b>Metabolism</b>\n
9,NaN,Metabolism,Global and overview maps,NaN,B Global and overview maps\n
10,01100,Metabolism,Global and overview maps,Metabolic pathways,C 01100 Metabolic pathways\n
11,01110,Metabolism,Global and overview maps,Biosynthesis of secondary metabolites,C 01110 Biosynthesis of secondary metaboli...
12,01120,Metabolism,Global and overview maps,Microbial metabolism in diverse environments,C 01120 Microbial metabolism in diverse en...
13,01130,Metabolism,Global and overview maps,Biosynthesis of antibiotics,C 01130 Biosynthesis of antibiotics\n
14,01200,Metabolism,Global and overview maps,Carbon metabolism,C 01200 Carbon metabolism\n
15,01210,Metabolism,Global and overview maps,2-Oxocarboxylic acid metabolism,C 01210 2-Oxocarboxylic acid metabolism\n
16,01212,Metabolism,Global and overview maps,Fatty acid metabolism,C 01212 Fatty acid metabolism\n
17,01230,Metabolism,Global and overview maps,Biosynthesis of amino acids,C 01230 Biosynthesis of amino acids\n
